<a href="https://colab.research.google.com/github/RicardoLelis/ds_nba/blob/main/ds_nba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %pip install nba_api

     |████████████████████████████████| 125 kB 4.3 MB/s 
  Created wheel for nba-api: filename=nba_api-1.1.11-py3-none-any.whl size=251504 sha256=a7708b8049595d2a515f106745d0abf19ad5a7847ffeb3358763b0ec049acab8
  Stored in directory: /root/.cache/pip/wheels/e3/f4/46/996128675381abc175e306642bae575d372dcf7efc4683e551
Successfully built nba-api


In [2]:
from nba_api.stats.endpoints import leaguegamefinder


In [7]:

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable='03/31/2022', league_id_nullable='00')
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

ReadTimeout: ignored

In [ ]:
games.columns

In [ ]:
games=games[['TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'PLUS_MINUS']]

In [ ]:
games

In [ ]:
# final dataframe:one row for one game
# two columns: 1.result of game: target 2.score stat comparing two teams: feature

In [ ]:
import pandas as pd

In [ ]:
games.info()

In [ ]:
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])

In [ ]:
games['GAME_DATE']

In [ ]:
games = games.sort_values('GAME_DATE')

In [ ]:
games

In [ ]:
games['avg_30_plus_minus']= games.groupby('TEAM_NAME')['PLUS_MINUS'].transform(lambda x: x.rolling(30, closed='left').mean())

In [ ]:
games[games['TEAM_NAME'] == "Toronto Raptors"].head(35)

In [ ]:
msk = games['MATCHUP'].str.contains('@')

In [ ]:
games_away = games[msk]
games_home = games[~msk]

In [ ]:
games_away.shape

In [ ]:
games_home.shape

In [ ]:
games_home

In [ ]:
games_away

In [ ]:
games_merged = pd.merge(games_home, games_away, on='GAME_ID', suffixes=('_home', '_away'))

In [ ]:
games_merged

In [ ]:
games_merged['avg_30_plus_minus_diff'] = games_merged['avg_30_plus_minus_home'] - games_merged['avg_30_plus_minus_away']

In [ ]:
games_merged

In [ ]:
games_model = games_merged[['WL_home', 'avg_30_plus_minus_diff']].dropna()

In [ ]:
games_model['WL_home'] = games_model['WL_home'].map({'W': 1, 'L': 0})

In [ ]:
games_model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(games_model)
